In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import plot_model

In [ ]:
df = pd.read_csv("Edge-IIoTset_112.csv", low_memory = False)

In [ ]:
df.info()

In [ ]:
print(df['Attack_type'].value_counts())

In [ ]:
# Creating a dictionary of Types
attacks = {'Normal': 0,'MITM': 1, 'Uploading': 2, 'Ransomware': 3, 'SQL_injection': 4,
       'DDoS_HTTP': 5, 'Password': 6,'Backdoor': 7, 'XSS': 8}
df['Attack_type'] = df['Attack_type'].map(attacks)

In [ ]:
X = df.drop(columns=['Attack_label', 'Attack_type'])
y = df['Attack_type']

In [ ]:
# Apply the Chi-Squared test
X = X.select_dtypes(include=[np.number])
chi_selector = SelectKBest(chi2, k='all')  # Set k to the desired number of features
X_kbest = chi_selector.fit_transform(X, y)

In [ ]:
# Get the scores for each feature
chi_scores = chi_selector.scores_

# Combine scores with feature names
chi_scores = pd.DataFrame({'feature': X.columns, 'score': chi_scores})

# Sort the features by their scores
chi_scores = chi_scores.sort_values(by='score', ascending=False)

print(chi_scores)

In [ ]:
selected_features = chi_scores['feature'].tolist()[:79]  # Select top k features

In [ ]:
# Split the data into train (80%) and test (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X[selected_features], y, test_size=0.2, random_state=42)

# Split the training data further into train (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [ ]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
def cnn_lstm_gru_model(input_shape, num_classes):
    
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),        
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        LSTM(64, return_sequences=True),
        GRU(64, return_sequences=False),
        
        Flatten(),
        
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
input_shape = (X_train.shape[1], 1)
num_classes = 9
model_cnn_lstm_gru = cnn_lstm_gru_model(input_shape, num_classes)
model_cnn_lstm_gru.summary()
plot_model(model_cnn_lstm_gru)

In [ ]:
train_start_time = time.time()
# Train the model
history_cnn_lstm_gru = model_cnn_lstm_gru.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)
# Record the ending time
train_end_time = time.time()

# Record the starting time for testing
test_start_time = time.time()
# Evaluate the model
loss, accuracy = model_cnn_lstm_gru.evaluate(X_test, y_test, batch_size=32)
# Record the ending time for testing
test_end_time = time.time()

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Calculate and print the training time
train_time = train_end_time - train_start_time
print(f"Training time: {train_time:.2f} seconds")

# Calculate and print the testing time
test_time = test_end_time - test_start_time
print(f"Testing time: {test_time:.2f} seconds")

In [ ]:
def cnn_lstm_bigru_model(input_shape, num_classes):
    
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),        
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        LSTM(64, return_sequences=True),
        Bidirectional(GRU(64, return_sequences=False)),
        
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
input_shape = (X_train.shape[1], 1)
num_classes = 9
model_cnn_lstm_bigru = cnn_lstm_bigru_model(input_shape, num_classes)
model_cnn_lstm_bigru.summary()
plot_model(model_cnn_lstm_bigru)

In [ ]:
train_start_time = time.time()
# Train the model
history_cnn_lstm_bigru = model_cnn_lstm_bigru.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)
# Record the ending time
train_end_time = time.time()

# Record the starting time for testing
test_start_time = time.time()
# Evaluate the model
loss, accuracy = model_cnn_lstm_bigru.evaluate(X_test, y_test, batch_size=32)
# Record the ending time for testing
test_end_time = time.time()

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Calculate and print the training time
train_time = train_end_time - train_start_time
print(f"Training time: {train_time:.2f} seconds")

# Calculate and print the testing time
test_time = test_end_time - test_start_time
print(f"Testing time: {test_time:.2f} seconds")

In [ ]:
def cnn_lstm_model(input_shape, num_classes):
    
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),        
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        
        LSTM(64, return_sequences=False),
        
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
input_shape = (X_train.shape[1], 1)
num_classes = 9
model_cnn_lstm = cnn_lstm_model(input_shape, num_classes)
model_cnn_lstm.summary()
plot_model(model_cnn_lstm)

In [ ]:
train_start_time = time.time()
# Train the model
history_cnn_lstm = model_cnn_lstm.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)
# Record the ending time
train_end_time = time.time()

# Record the starting time for testing
test_start_time = time.time()
# Evaluate the model
loss, accuracy = model_cnn_lstm.evaluate(X_test, y_test, batch_size=32)
# Record the ending time for testing
test_end_time = time.time()

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Calculate and print the training time
train_time = train_end_time - train_start_time
print(f"Training time: {train_time:.2f} seconds")

# Calculate and print the testing time
test_time = test_end_time - test_start_time
print(f"Testing time: {test_time:.2f} seconds")

In [ ]:
# Plot the training and validation accuracy over the epochs

# CNN-LSTM-GRU
plt.plot(history_cnn_lstm_gru.history['accuracy'])
plt.plot(history_cnn_lstm_gru.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# CNN-LSTM-BiGRU
plt.plot(history_cnn_lstm_bigru.history['accuracy'])
plt.plot(history_cnn_lstm_bigru.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# CNN-LSTM
plt.plot(history_cnn_lstm.history['accuracy'])
plt.plot(history_cnn_lstm.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Plot the training and validation loss over the epochs

# CNN-LSTM-GRU
plt.plot(history_cnn_lstm_gru.history['loss'])
plt.plot(history_cnn_lstm_gru.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# CNN-LSTM-BiGRU
plt.plot(history_cnn_lstm_bigru.history['loss'])
plt.plot(history_cnn_lstm_bigru.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# CNN-LSTM
plt.plot(history_cnn_lstm.history['loss'])
plt.plot(history_cnn_lstm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Make predictions for each model
y_pred_cnn_lstm_gru = model_cnn_lstm_gru.predict(np.expand_dims(X_test, axis=2))
y_pred_cnn_lstm_bigru = model_cnn_lstm_bigru.predict(np.expand_dims(X_test, axis=2))
y_pred_cnn_lstm = model_cnn_lstm.predict(np.expand_dims(X_test, axis=2))

In [ ]:
# CNN-LSTM-GRU------------------------------------------------------------------------------------------------------------------------------------------------


y_pred_classes_cnn_lstm_gru = np.argmax(y_pred_cnn_lstm_gru, axis=1)
# Inverse the 'attacks' dictionary to map back numbers to names
inverse_attacks = {v: k for k, v in attacks.items()}
# Generate the classification report with attack names instead of numbers
class_report = classification_report(y_test, y_pred_classes_cnn_lstm_gru, target_names=[inverse_attacks[i] for i in range(len(inverse_attacks))])
print(class_report)


# CNN-LSTM-BiGRU------------------------------------------------------------------------------------------------------------------------------------------------


y_pred_classes_cnn_lstm_bigru = np.argmax(y_pred_cnn_lstm_bigru, axis=1)
# Inverse the 'attacks' dictionary to map back numbers to names
inverse_attacks = {v: k for k, v in attacks.items()}
# Generate the classification report with attack names instead of numbers
class_report = classification_report(y_test, y_pred_classes_cnn_lstm_bigru, target_names=[inverse_attacks[i] for i in range(len(inverse_attacks))])
print(class_report)


# CNN-LSTM------------------------------------------------------------------------------------------------------------------------------------------------


y_pred_classes_cnn_lstm = np.argmax(y_pred_cnn_lstm, axis=1)
# Inverse the 'attacks' dictionary to map back numbers to names
inverse_attacks = {v: k for k, v in attacks.items()}
# Generate the classification report with attack names instead of numbers
class_report = classification_report(y_test, y_pred_classes_cnn_lstm, target_names=[inverse_attacks[i] for i in range(len(inverse_attacks))])
print(class_report)

In [ ]:
# CNN-LSTM-GRU------------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm_gru)
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM-GRU Confusion Matrix')
plt.show()


# CNN-LSTM-BiGRU------------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm_bigru)
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM-BiGRU Confusion Matrix')
plt.show()


# CNN-LSTM------------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm)
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM Confusion Matrix')
plt.show()

In [ ]:
# CNN-LSTM-GRU-----------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm_gru)
# Normalize the confusion matrix by dividing each value by the sum of its row (i.e., the number of true instances for each label)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat_normalized, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM-GRU Normalized Confusion Matrix as Percentages')
plt.show()


# CNN-LSTM-BiGRU-----------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm_bigru)
# Normalize the confusion matrix by dividing each value by the sum of its row (i.e., the number of true instances for each label)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat_normalized, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM-BiGRU Normalized Confusion Matrix as Percentages')
plt.show()


# CNN-LSTM-----------------------------------------------------------------------------------------------------------------------------------------------


# Compute the confusion matrix
conf_mat = confusion_matrix(y_test, y_pred_classes_cnn_lstm)
# Normalize the confusion matrix by dividing each value by the sum of its row (i.e., the number of true instances for each label)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]
# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat_normalized, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('CNN-LSTM Normalized Confusion Matrix as Percentages')
plt.show()

In [ ]:
num_classes = 9

# Get predictions for training sets
pred_cnn_lstm_bigru_train = model_cnn_lstm_bigru.predict(np.expand_dims(X_train, axis=2))
pred_cnn_lstm_gru_train = model_cnn_lstm_gru.predict(np.expand_dims(X_train, axis=2))

# Get predictions for validation sets
pred_cnn_lstm_bigru_val = model_cnn_lstm_bigru.predict(np.expand_dims(X_val, axis=2))
pred_cnn_lstm_gru_val = model_cnn_lstm_gru.predict(np.expand_dims(X_val, axis=2))

# Concatenate predictions for validation sets
X_conc_val = np.concatenate([pred_cnn_lstm_bigru_val, pred_cnn_lstm_gru_val], axis=1)

# Concatenate predictions for training sets
X_conc_train = np.concatenate([pred_cnn_lstm_bigru_train, pred_cnn_lstm_gru_train], axis=1)

# Concatenate predictions for test sets
X_conc_test = np.concatenate([y_pred_cnn_lstm_bigru, y_pred_cnn_lstm_gru], axis=1)


# Define the ensemble model
ensemble_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_conc_train.shape[1],)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

ensemble_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the ensemble model

train_start_time = time.time()

ensemble_model.fit(X_conc_train, y_train, validation_data=(X_conc_val, y_val), epochs=50, batch_size=32)

# Record the ending time
train_end_time = time.time()

# Record the starting time for testing
test_start_time = time.time()
# Evaluate the model
loss, accuracy = ensemble_model.evaluate(X_conc_test, y_test, batch_size=32)
# Record the ending time for testing
test_end_time = time.time()

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Calculate and print the training time
train_time = train_end_time - train_start_time
print(f"Training time: {train_time:.2f} seconds")

# Calculate and print the testing time
test_time = test_end_time - test_start_time
print(f"Testing time: {test_time:.2f} seconds")

In [ ]:
# Get predicted class labels
final_pred = np.argmax(ensemble_model.predict(X_conc_test), axis=1)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, final_pred))

# confusion matrix
conf_mat = confusion_matrix(y_test, final_pred)

# Normalize the confusion matrix by dividing each value by the sum of its row (i.e., the number of true instances for each label)
conf_mat_normalized = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]

# Convert the 'attacks' dictionary to a list of class names ordered by the class number
class_names_ordered = [attack for attack, number in sorted(attacks.items(), key=lambda item: item[1])]

# Plot the heatmap using seaborn
plt.figure(figsize=(15, 10))
sns.heatmap(conf_mat_normalized, annot=True, fmt=".2%", cmap="Blues", xticklabels=class_names_ordered, yticklabels=class_names_ordered)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Ensemble Model Normalized Confusion Matrix as Percentages')
plt.show()